# M4L3: Data Munging & Validation

## `pandas` for Data Munging

`pandas` can be very helpful for cleaning up tabular data.

Because `pandas` has a lot of functionality, it can be a bit overwhelming to learn.  We'll walk through an example together to illustrate some of the most useful features for data cleaning.

### Refresher: `pandas` Data Structures

* `Series` - a one-dimensional array of data. Each element has an index (like a dictionary key), and a value.
* `DataFrame` - a two-dimensional array of data, each column is a `Series`.

Another way to frame our understanding of the DataFrame object is to consider it a specialized dictionary. Whereas a dictionary maps arbitrary keys to arbitrary values, a DataFrame maps a column name to a Series of column data.

In [ ]:
votes = pd.DataFrame(
    {'yes_votes': {'Aukerman': 42, 'Mantzoukas': 54, 'Tompkins': 414, 'Haddad': 90},
    'no_votes': {'Aukerman': 42, 'Mantzoukas': 12, 'Tompkins': 601, 'Haddad': 30}
    },
)
votes

In [ ]:
# can calculate new columns
votes['yes_pct'] = votes.yes_votes / (votes.yes_votes + votes.no_votes)
votes

In [ ]:
# index by column
votes["yes_pct"]

In [ ]:
# or by row
votes.loc['Aukerman']

<https://pandas.pydata.org/docs/reference/frame.html>

### Reading Data

https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

```python

import pandas as pd

# read a csv file
df = pd.read_csv("data.csv")

# read from a database
conn = sqlite3.connect("data.db")
df = pd.read_sql("SELECT * FROM data", conn)
```

### Aside: How does `read_sql` work?

### Useful Methods for Exploring Data

* `df.head()` - show the first few rows of the DataFrame
* `df.tail()` - show the last few rows of the DataFrame
* `df.info()` - show the column names, data types, and number of non-null values
* `df.describe()` - show summary statistics for numeric columns
* `df.columns` - show the column names
* `df.dtypes` - show the data types of each column

Working with Missing Data: https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html

### Useful Methods for Cleaning Data

* `df.dropna()` - drop rows with null values
* `df.fillna()` - replace null values with a value
* `df.drop()` - drop rows or columns according to a label or index
* `df.rename()` - rename columns
* `df.astype()` - change the data type of a column

String Operations: <https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html>

### Example: Loading Crime Data

(Adapted from http://vis.stanford.edu/wrangler/)

In [ ]:
import pandas as pd

crime = pd.read_csv("crime.csv", header=None)
print(crime)

In [ ]:
# we can provide our own name for the columns
crime.columns = ["year", "crimerate"]
crime

In [ ]:
# we can build an index of the rows
crime.year.isnull() & crime.crimerate.isnull()

In [ ]:
# and drop these rows with `.drop`
crime.drop(crime.index[crime.year.isnull() & crime.crimerate.isnull()], inplace=True)
crime

In [ ]:
crime.year

In [ ]:
crime.crimerate

In [ ]:
# we can add a new column
crime["state"] = crime.year.str.extract("in\s+(.*)")
crime

In [ ]:
# and replace null values
crime.state.fillna(method='ffill', inplace=True)
crime

In [ ]:
# now can drop all rows that still have nulls
crime.dropna(inplace=True)
crime

In [ ]:
# and pivot the data
crime = crime.pivot(index="state", columns="year", values="crimerate")
crime

In [ ]:
# the data when formatted this way is suitable for
# plotting with matplotlib
crime.loc["Illinois"].plot(kind="bar")

### Golden Rules of `pandas`

**Avoid using `for` loops when possible.  `pandas` is designed to work with vectorized operations.**

If you need to do something to every row or column, first go to the `pandas` documentation and see if there is a built-in method for doing it.

If there isn't, you can use `applymap` to apply a function to every row or column as a last resort.

In [ ]:
# e.g. we can double every value in the DataFrame
crime.applymap(lambda x: x * 2)

# or without lambda, equivalent to this
def double(x):
    return x * 2

crime.applymap(double)

## General Purpose Validation

Tip #1) When validating numeric types, convert using `int()` or `float()` and catch `ValueError`.

```python
value = int("123")

try:
    value = int("123x")
except ValueError:
    print("invalid value")
```

Tip #2) Consider the maxim "be liberal in what you accept, and conservative in what you send."

e.g. If your program is reading a column as a boolean, you may want to accept "true", "TRUE", "t", "F", "False", etc.
But when you write a boolean value, you should only write "True" or "False".  (or whatever the standard is for your program)

Tip #3) When validating dates, use `datetime.strptime()` and catch `ValueError`.

```python
from datetime import datetime

value = datetime.strptime("2020-01-01", "%Y-%m-%d")

try:
    value = datetime.strptime("2020-01-01x", "%Y-%m-%d")
except ValueError:
    print("invalid value")
```

Tip #4) Lean on the standard library. It has a lot of useful functions for validating data.

* `email.utils.parseaddr()`
* `ipaddress.ip_address()`
* `urllib.parse.urlparse()`
* `json.loads()`

Tip #5) When validating other strings, do cleanup first (e.g. `strip()`, `lower()`, etc.). Then use `isalpha()`, `isdigit()`, `isalnum()`, etc. or regular expressions.


## `namedtuple`, `dataclasses`, and `pydantic`

Python being a loosely typed language can increase the likelihood of certain types of bugs in your code.

```python

def get_user_info(user_id):
    # fetch user information from an API or database
    ...

    return {
        "id": user_id,
        "name": name,
        "email": email,
        "age": age,
        "address": address,
        "phone": phone,
    }

user = get_user_info(123)
```

This works fine, but we've discussed how it can be useful to have a defined interface. The only way to know what keys to expect in a user dictionary is to read the documentation for `get_user_info`.

You might be tempted to write a `User` class, just to have a pre-defined set of fields.

There are other options, though. Python has a few built-in data structures that can help you define a data structure without having to write a class.

### `namedtuple`

Often you have a pre-defined set of fields, and you want to be able to access them by name.

`namedtuple` is perfect for this use case.  It is exactly what it sounds like, a tuple with named fields.

```python
from collections import namedtuple

# define the namedtuple type 
# (this would be done at the global scope so it can be used within your module)
User = namedtuple("User", ["id", "name", "email", "age", "address", "phone"])

# you can then use User the way you would a class
def get_user_info(user_id):
    # fetch user information from an API or database
    ...

    # same as order in definition
    return User(user_id, name, email, age, address, phone)

user = get_user_info(123)
# fields can be accessed like attributes
print(user.name)
# or like a tuple
user[0] == user.id
```

`namedtuple`s are immutable, so you can't change the values of the fields after they are created.

A `namedtuple` is useful anywhere a tuple was being used.

Think back to your `Hashtable` implementation.
Your list of key-value pairs was a list of tuples.

You likely had code like:

```python
while items[index] is not None:
    if key == items[index][0]:
        return value

    index = (index + 1) % self.capacity
```

Do you remember at a glance what that `[0]` was? Possibly not. 
If you had a `namedtuple` for your key-value pairs, you could have written:

```python
while items[index] is not None:
    if key == items[index].key:
        return value

    index = (index + 1) % self.capacity
```

assuming a `namedtuple` definition like:

```python
KeyValuePair = namedtuple("KeyValuePair", ["key", "value"])
```

`namedtuple`: <https://docs.python.org/3/library/collections.html#collections.namedtuple>

In PA #4: https://github.com/uchicago-capp122-spring23/pa4-jamesturk/blob/main/linkage/datatypes.py

### `dataclasses`

`dataclasses` are a newer addition to Python, and are a bit more powerful than `namedtuple`s.

They also take advantage of some newer Python syntax you may not have seen before called type annotations.

```python
from dataclasses import dataclass

@dataclass
class User:
    id: int
    name: str
    email: str
    age: int
    address: str
    phone: str
```

This creates a class with the same fields as the `namedtuple` example above.

A default `__init__` method is created for you, so you can create a `User` instance like:

```python
user = User(123, "Kulap", "kulap@hotmail.com", 40, "123 Main St", "555-555-5555")

# unlike a namedtuple, you can change the values of the fields
user.name = "Kulap Vilaysack"
```

`dataclasses`: <https://docs.python.org/3/library/dataclasses.html>

type annotations: <https://docs.python.org/3/library/typing.html>

### `pydantic`

Taking this a step further, you can use a library called `pydantic` to define a data structure and validate the data that is passed to it.

```python
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str
    age: int
    address: str
    phone: str
```

This creates a class with the same fields as the `dataclass` example above.

But we can also add validation to the fields.

In [ ]:
from pydantic import BaseModel, EmailStr, validator

class User(BaseModel):
    id: int
    name: str
    email: EmailStr
    age: int
    address: str
    phone: str

    @validator("age")
    def age_must_be_positive(cls, value):
        if value < 0:
            raise ValueError("age must be positive")
        return value

This creates a class where assigning invalid values to attributes raises errors.

In [ ]:
user = User(
        id=123,
        name="Scott",
        email="not-an-email",
        age=-4,
        address="123 Main St",
        phone="555-555-5555")

pydantic: <https://docs.pydantic.dev>
